# PART C

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
from math import log

# Función para cargar el dataset
def cargar_dataset(ruta_csv, nrows):
    # Saltar la primera fila usando skiprows=1
    df = pd.read_csv(ruta_csv, sep=';', skiprows=1, header=None, names=["id", "text", "date", "label"], low_memory=False, nrows=nrows)
    
    # Convertir la columna 'label' a tipo int
    df["label"] = pd.to_numeric(df["label"], errors='coerce') 
    df = df.dropna(subset=["text", "label"]).reset_index(drop=True)
    df["label"] = df["label"].astype(int)
    return df

# División balanceada en train y test
def dividir_train_test(df, test_size=0.2, seed=42):
    from sklearn.model_selection import train_test_split
    train_df, test_df = train_test_split(df, test_size=test_size, random_state=seed, stratify=df['label'])
    return train_df, test_df

# Construir diccionario y estadísticas
def construir_diccionario(train_df, min_freq=1):
    pos_texts = train_df[train_df['label'] == 1]['text']
    neg_texts = train_df[train_df['label'] == 0]['text']

    word_counts_pos = Counter()
    word_counts_neg = Counter()

    for text in pos_texts:
        word_counts_pos.update(text.split())

    for text in neg_texts:
        word_counts_neg.update(text.split())

    # Reducir diccionario eliminando palabras con frecuencia menor a min_freq (si lo necesitaras)
    if min_freq > 1:
        word_counts_pos = Counter({k: v for k, v in word_counts_pos.items() if v >= min_freq})
        word_counts_neg = Counter({k: v for k, v in word_counts_neg.items() if v >= min_freq})

    total_tweets = len(train_df)
    p_pos = len(pos_texts) / total_tweets
    p_neg = len(neg_texts) / total_tweets

    return {
        'word_counts_pos': word_counts_pos,
        'word_counts_neg': word_counts_neg,
        'p_pos': p_pos,
        'p_neg': p_neg,
        'vocab_size': len(set(word_counts_pos.keys()).union(set(word_counts_neg.keys())))
    }

# Clasificación de un tweet usando Naïve Bayes sin Laplace Smoothing
def predecir_tweet_optimizado(tweet, stats):
    log_prob_pos = log(stats['p_pos'])
    log_prob_neg = log(stats['p_neg'])
    vocab_size = stats['vocab_size']

    words_pos = stats['word_counts_pos']
    words_neg = stats['word_counts_neg']

    total_words_pos = sum(words_pos.values())
    total_words_neg = sum(words_neg.values())

    words = tweet.split()

    for word in words:
        w_count_pos = words_pos.get(word, 0)
        w_count_neg = words_neg.get(word, 0)

        # Sin Laplace Smoothing:
        # Si w_count_pos = 0 => P(w|pos) = 0 => log_prob_pos = -inf
        if w_count_pos > 0:
            log_prob_pos += log(w_count_pos / total_words_pos)
        else:
            log_prob_pos = float('-inf')

        # Si w_count_neg = 0 => P(w|neg) = 0 => log_prob_neg = -inf
        if w_count_neg > 0:
            log_prob_neg += log(w_count_neg / total_words_neg)
        else:
            log_prob_neg = float('-inf')

    return 1 if log_prob_pos > log_prob_neg else 0

# Evaluación del modelo
def evaluar_modelo_optimizado(test_df, stats):
    y_pred = test_df['text'].apply(lambda tweet: predecir_tweet_optimizado(tweet, stats))
    y_true = test_df['label'].values
    accuracy = np.mean(y_pred == y_true)
    print("accuracy del modelo:", accuracy)
    return accuracy

# Main
if __name__ == "__main__":
    ruta_csv = "./CRI_Practica3/FinalStemmedSentimentAnalysisDataset.csv"
    df = cargar_dataset(ruta_csv, None) # Segundo argumento = numero de filas a usar
    print("Ejemplo de datos:\n", df.head())

    # División en train y test
    train_df, test_df = dividir_train_test(df)
    print(f"\nTamaño del conjunto de entrenamiento: {len(train_df)}")
    print(f"Tamaño del conjunto de prueba: {len(test_df)}")

    # Construir diccionario y estadísticas (sin Laplace)
    stats = construir_diccionario(train_df)

    # Evaluar el modelo (sin Laplace)
    evaluar_modelo_optimizado(test_df, stats)
   


Ejemplo de datos:
    id                                               text        date  label
0   1                       is so sad for my apl friend   04/03/2015      0
1   2                         i miss the new moon trail   06/10/2015      0
2   3                              omg it already 730 o   03/04/2015      1
3   4   omgag im sooo im gunn cry i've been at thi de...  13/11/2015      0
4   5                     i think mi bf is che on me tt   10/08/2015      0

Tamaño del conjunto de entrenamiento: 1251424
Tamaño del conjunto de prueba: 312856
accuracy del modelo: 0.662576392973125


# PART B

In [37]:

# Evaluación del modelo
def evaluar_modelo_optimizado(test_df, stats):
    y_pred = test_df['text'].apply(lambda tweet: predecir_tweet_optimizado(tweet, stats))
    y_true = test_df['label'].values
    accuracy = np.mean(y_pred == y_true)
    return accuracy

# Estrategia 1: Aumentar el tamaño del conjunto de entrenamiento
def estrategia_1(df, incrementos):
    print("\nEstrategia 1: Aumentar tamaño de entrenamiento (diccionario variable)")
    resultados = []
    for n in incrementos:
        train_df, test_df = dividir_train_test(df[:n])
        stats = construir_diccionario(train_df)
        acc = evaluar_modelo_optimizado(test_df, stats)
        resultados.append((n, acc))
        print(f"Tamaño Train: {n}, accuracy: {acc:.4f}")
    return resultados

# Estrategia 2: Reducir el diccionario manteniendo el train fijo
def estrategia_2(df, min_freqs):
    print("\nEstrategia 2: Modificar tamaño del diccionario")
    train_df, test_df = dividir_train_test(df)
    resultados = []
    for min_freq in min_freqs:
        stats = construir_diccionario(train_df, min_freq=min_freq)
        acc = evaluar_modelo_optimizado(test_df, stats)
        resultados.append((min_freq, acc))
        print(f"Frecuencia mínima: {min_freq}, accuracy: {acc:.4f}")
    return resultados

# Estrategia 3: Mantener el tamaño del diccionario fijo y variar el tamaño del conjunto de entrenamiento
def estrategia_3(df, train_sizes):
    print("\nEstrategia 3: Modificar tamaño del conjunto de entrenamiento (diccionario fijo)")
    # Dividir una sola vez para obtener test fijo
    train_df, test_df = dividir_train_test(df, test_size=0.2)
    # Construir diccionario con el train completo
    stats_fijo = construir_diccionario(train_df)
    resultados = []
    for size in train_sizes:
        train_subset = train_df[:size]
        # Aquí se recalcula el diccionario con el subset, como en la versión original
        stats = construir_diccionario(train_subset)
        acc = evaluar_modelo_optimizado(test_df, stats)
        resultados.append((size, acc))
        print(f"Tamaño Train: {size}, accuracy: {acc:.4f}")
    return resultados

# Main
if __name__ == "__main__":
    ruta_csv = "./CRI_Practica3/FinalStemmedSentimentAnalysisDataset.csv"
    print("Parte B: Estrategias (Sin Laplace Smoothing)")
    df = cargar_dataset(ruta_csv, 100000)

    # Estrategia 1
    estrategia_1(df, incrementos=[20000, 40000, 60000, 80000])

    # Estrategia 2
    estrategia_2(df, min_freqs=[1, 3, 5, 10])

    # Estrategia 3
    estrategia_3(df, train_sizes=[20000, 40000, 60000, 80000])


Parte B: Estrategias (Sin Laplace Smoothing)

Estrategia 1: Aumentar tamaño de entrenamiento (diccionario variable)
Tamaño Train: 20000, accuracy: 0.5915
Tamaño Train: 40000, accuracy: 0.5641
Tamaño Train: 60000, accuracy: 0.5598
Tamaño Train: 80000, accuracy: 0.5528

Estrategia 2: Modificar tamaño del diccionario
Frecuencia mínima: 1, accuracy: 0.5500
Frecuencia mínima: 3, accuracy: 0.4971
Frecuencia mínima: 5, accuracy: 0.4770
Frecuencia mínima: 10, accuracy: 0.4587

Estrategia 3: Modificar tamaño del conjunto de entrenamiento (diccionario fijo)
Tamaño Train: 20000, accuracy: 0.4940
Tamaño Train: 40000, accuracy: 0.5195
Tamaño Train: 60000, accuracy: 0.5359
Tamaño Train: 80000, accuracy: 0.5500


# PART A

In [38]:
import pandas as pd
import numpy as np
from collections import Counter
from math import log

# Asumimos que ya tenemos las funciones cargar_dataset, dividir_train_test, construir_diccionario y predecir_tweet definidas.
# También asumimos que predecir_tweet implementa Laplace Smoothing.

def predecir_tweet(tweet, stats, alpha=1):
    log_prob_pos = log(stats['p_pos'])
    log_prob_neg = log(stats['p_neg'])
    vocab_size = stats['vocab_size']
    words = tweet.split()

    words_pos = stats['word_counts_pos']
    words_neg = stats['word_counts_neg']

    total_words_pos = sum(words_pos.values())
    total_words_neg = sum(words_neg.values())

    for word in words:
        p_w_pos = (words_pos.get(word, 0) + alpha) / (total_words_pos + alpha * vocab_size)
        p_w_neg = (words_neg.get(word, 0) + alpha) / (total_words_neg + alpha * vocab_size)
        log_prob_pos += log(p_w_pos)
        log_prob_neg += log(p_w_neg)

    return 1 if log_prob_pos > log_prob_neg else 0

def evaluar_modelo(test_df, stats, alpha=1):
    y_pred = test_df['text'].apply(lambda tweet: predecir_tweet(tweet, stats, alpha=alpha))
    y_true = test_df['label'].values
    return np.mean(y_pred == y_true)

# Estrategia 1 con Laplace Smoothing: Aumentar el tamaño de entrenamiento
def estrategia_1_laplace(df, incrementos, alpha=1):
    print("\n[Estrategia 1 - Parte A]: Aumentar tamaño de entrenamiento con Laplace Smoothing")
    for n in incrementos:
        train_df, test_df = dividir_train_test(df[:n])
        stats = construir_diccionario(train_df)
        acc = evaluar_modelo(test_df, stats, alpha=alpha)
        print(f"Tamaño Train: {n}, alpha: {alpha}, accuracy: {acc:.4f}")

# Estrategia 2 con Laplace Smoothing: Fijar train, variar tamaño del diccionario
def estrategia_2_laplace(df, min_freqs, alpha=1):
    print("\n[Estrategia 2 - Parte A]: Variar tamaño del diccionario con Laplace Smoothing (train fijo)")
    train_df, test_df = dividir_train_test(df)
    for min_freq in min_freqs:
        stats = construir_diccionario(train_df, min_freq=min_freq)
        acc = evaluar_modelo(test_df, stats, alpha=alpha)
        print(f"Frecuencia mínima: {min_freq}, alpha: {alpha}, accuracy: {acc:.4f}")

# Estrategia 3 con Laplace Smoothing: Fijar diccionario, variar tamaño del entrenamiento
def estrategia_3_laplace(df, train_sizes, alpha=1):
    print("\n[Estrategia 3 - Parte A]: Mantener diccionario fijo y variar tamaño de entrenamiento con Laplace Smoothing")
    # Primero creamos un conjunto de entrenamiento y test fijos
    train_full_df, test_df = dividir_train_test(df)
    # Creamos un diccionario con todo el train, que será nuestro diccionario base
    stats_full = construir_diccionario(train_full_df)
    full_vocab = set(stats_full['word_counts_pos'].keys()).union(set(stats_full['word_counts_neg'].keys()))
    vocab_size = stats_full['vocab_size']

    for size in train_sizes:
        train_subset = train_full_df[:size]
        # Recalculamos las frecuencias solo considerando las palabras del vocab completo
        pos_texts = train_subset[train_subset['label'] == 1]['text']
        neg_texts = train_subset[train_subset['label'] == 0]['text']
        
        word_counts_pos = Counter()
        word_counts_neg = Counter()
        
        for text in pos_texts:
            word_counts_pos.update([w for w in text.split() if w in full_vocab])
        for text in neg_texts:
            word_counts_neg.update([w for w in text.split() if w in full_vocab])
        
        p_pos = len(pos_texts) / len(train_subset)
        p_neg = len(neg_texts) / len(train_subset)
        
        # Ahora creamos un stats con el vocab_size fijo
        stats_subset = {
            'word_counts_pos': word_counts_pos,
            'word_counts_neg': word_counts_neg,
            'p_pos': p_pos,
            'p_neg': p_neg,
            'vocab_size': vocab_size
        }
        
        acc = evaluar_modelo(test_df, stats_subset, alpha=alpha)
        print(f"Tamaño Train: {size}, alpha: {alpha}, accuracy: {acc:.4f}")

# Main para la Parte A
if __name__ == "__main__":
    # Cargamos el dataset (definir cargar_dataset, dividir_train_test, construir_diccionario previamente)
    ruta_csv = "./CRI_Practica3/FinalStemmedSentimentAnalysisDataset.csv"
    df = cargar_dataset(ruta_csv, nrows=100000)
    
    # Parámetros para probar
    incrementos = [20000, 40000, 60000, 80000]
    min_freqs = [1, 3, 5]
    train_sizes = [20000, 40000, 60000, 80000]
    alpha = 1

    # Estrategia 1 con Laplace
    estrategia_1_laplace(df, incrementos, alpha=alpha)

    # Estrategia 2 con Laplace
    estrategia_2_laplace(df, min_freqs, alpha=alpha)

    # Estrategia 3 con Laplace
    estrategia_3_laplace(df, train_sizes, alpha=alpha)



[Estrategia 1 - Parte A]: Aumentar tamaño de entrenamiento con Laplace Smoothing
Tamaño Train: 20000, alpha: 1, accuracy: 0.7522
Tamaño Train: 40000, alpha: 1, accuracy: 0.7508
Tamaño Train: 60000, alpha: 1, accuracy: 0.7614
Tamaño Train: 80000, alpha: 1, accuracy: 0.7616

[Estrategia 2 - Parte A]: Variar tamaño del diccionario con Laplace Smoothing (train fijo)
Min. Frecuencia: 1, alpha: 1, accuracy: 0.7597
Min. Frecuencia: 3, alpha: 1, accuracy: 0.7514
Min. Frecuencia: 5, alpha: 1, accuracy: 0.7462

[Estrategia 3 - Parte A]: Mantener diccionario fijo y variar tamaño de entrenamiento con Laplace Smoothing
Tamaño Train: 20000, alpha: 1, accuracy: 0.7406
Tamaño Train: 40000, alpha: 1, accuracy: 0.7533
Tamaño Train: 60000, alpha: 1, accuracy: 0.7562
Tamaño Train: 80000, alpha: 1, accuracy: 0.7597
